# Lab 6: Vector Embeddings
So far, we've dealt primarily with real-valued input variables, like (x, y)-coordinates or pixel colors.
But lots of interesting input types, like text and location, are discrete.
Vector embeddings are the preferred way to deal with discrete inputs -- they're satisfying in theory, work well in practice, and have some really cool properties.

# The problem with discrete variables
TODO

## Try 1: Integer encoding
TODO

## Try 2: One-hot encoding
TODO

# Vector embeddings
TODO

### Wait, aren't we just one-hot encoding again?
TODO

# Distributed representation
TODO

# Sampled loss functions
TODO

# Embeddings in TensorFlow
TODO

# Embeddings in Keras
TODO

# Visualize embeddings with TensorBoard
TODO

# Example: ???
TODO